In [154]:
# things we need for NLP
import nltk
#nltk.download('punkt')

In [155]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [156]:
# things we need for Tensorflow
import numpy as np

In [157]:
import tflearn
import tensorflow as tf
import random

In [158]:
# import our chat-bot intents file
import json
with open('intents_ch.json') as json_data:
    intents = json.load(json_data)

In [159]:
import jieba
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        #w = nltk.word_tokenize(pattern)
        w = jieba.cut_for_search(pattern)
        w = ("/".join(w)).split("/")
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        #documents.append((intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))
print('documents:',documents)
print('------')
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

documents: [(['你好'], '问候'), (['嗨'], '问候'), (['机器', '机器人'], '问候'), (['你好'], '问候'), (['再见'], '再见'), (['待', '会见'], '再见'), (['拜拜'], '再见'), (['谢谢'], '谢谢'), (['谢谢', '你'], '谢谢'), (['太', '谢谢', '你', '了'], '谢谢'), (['你们', '每天', '几点', '开门', '啊'], '时间'), (['你们', '营业', '时间', '营业时间', '是', '是', '几点'], '时间'), (['时候', '啥时候', '开门'], '时间'), (['你们', '有', '什么', '助力', '助力车'], '助力车'), (['有', '什么', '什么样', '的', '助力', '助力车'], '助力车'), (['你们', '出租', '什么'], '助力车'), (['你们', '刷', '信用', '信用卡', '吗'], '付款'), (['刷卡', '吗'], '付款'), (['你们', '收', '万事', '达卡', '万事达', '万事达卡', '吗'], '付款'), (['只能', '现金', '支付', '现金支付'], '付款'), (['什么', '时候', '开门'], '营业时间'), (['你们', '几点', '开门'], '营业时间'), (['你们', '营业', '时间', '营业时间', '是', '什么', '时候'], '营业时间'), (['我们', '可以', '租个', '助力', '助力车', '吗'], '出租'), (['我想', '租个', '助力', '助力车'], '出租'), (['今天'], '今天租'), (['明天'], '明天租'), (['你们', '哪', '天', '方便'], '你们哪天方便'), (['你们', '哪', '天', '可以'], '你们哪天方便')]
------
29 documents
11 classes ['今天租', '付款', '你们哪天方便', '再见', '出租', '助力车', '时间', '明天租', '营业时间', '谢谢', '问候']
53

In [160]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    #pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    print(pattern_words)
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


['你好']
['嗨']
['机器', '机器人']
['你好']
['再见']
['待', '会见']
['拜拜']
['谢谢']
['谢谢', '你']
['太', '谢谢', '你', '了']
['你们', '每天', '几点', '开门', '啊']
['你们', '营业', '时间', '营业时间', '是', '是', '几点']
['时候', '啥时候', '开门']
['你们', '有', '什么', '助力', '助力车']
['有', '什么', '什么样', '的', '助力', '助力车']
['你们', '出租', '什么']
['你们', '刷', '信用', '信用卡', '吗']
['刷卡', '吗']
['你们', '收', '万事', '达卡', '万事达', '万事达卡', '吗']
['只能', '现金', '支付', '现金支付']
['什么', '时候', '开门']
['你们', '几点', '开门']
['你们', '营业', '时间', '营业时间', '是', '什么', '时候']
['我们', '可以', '租个', '助力', '助力车', '吗']
['我想', '租个', '助力', '助力车']
['今天']
['明天']
['你们', '哪', '天', '方便']
['你们', '哪', '天', '可以']


In [161]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=2000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 7999  | total loss: 0.25392 | time: 0.028s
| Adam | epoch: 2000 | loss: 0.25392 - acc: 0.9820 -- iter: 24/29
Training Step: 8000  | total loss: 0.21116 | time: 0.037s
| Adam | epoch: 2000 | loss: 0.21116 - acc: 0.9838 -- iter: 29/29
--
INFO:tensorflow:/home/mqm/Workspace/dl/notebooks/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [166]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [167]:
p = bow("刷卡", words,True)
print (p)
print (classes)

found in bag: 刷卡
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['今天租', '付款', '你们哪天方便', '再见', '出租', '助力车', '时间', '明天租', '营业时间', '谢谢', '问候']


In [168]:
print(model.predict([p]))

[[  1.97620247e-03   6.55262470e-01   6.65306152e-06   9.32808146e-02
    2.09343396e-02   1.56371112e-04   2.03908961e-02   1.51915636e-04
    1.45199872e-03   4.14905488e-04   2.05973402e-01]]


In [169]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )